In [ ]:
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from scipy import signal
import numpy as np
import librosa
import librosa.display
import wave
import random as rn
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive",)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import sys
os.chdir('/content/gdrive/MyDrive/datasets/recordings')
!pwd
ROOT_DIR = os.path.abspath("")
print (ROOT_DIR)
sys.path.append(ROOT_DIR)

/content/gdrive/MyDrive/datasets/recordings
/content/gdrive/MyDrive/datasets/recordings


In [ ]:
filenames = []

for flist in os.listdir(ROOT_DIR):
  filenames.append(flist)
rn.shuffle(filenames)

In [ ]:
test_speaker = 'theo'
train_mfccs = []
train_y = []
test_mfccs = []
test_y = []
pad2d = lambda a, i: a[:, 0: i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

In [ ]:
for i in range(len(filenames)):
  struct = filenames[i].split('_')
  digit = struct[0]
  speaker = struct[1]
  wav, sr = librosa.load(os.path.join(ROOT_DIR, filenames[i]))
  mfcc = librosa.feature.mfcc(wav)
  padded_mfcc = pad2d(mfcc, 40)
  if speaker == test_speaker:
    test_mfccs.append(padded_mfcc.flatten())
    test_y.append(int(digit))
  else:
    train_mfccs.append(padded_mfcc.flatten())
    train_y.append(int(digit))

In [ ]:
train_mfccs = np.array(train_mfccs)
test_mfccs = np.array(test_mfccs)

train_X_ex = np.expand_dims(train_mfccs, -1)
test_X_ex = np.expand_dims(test_mfccs, -1)

train_X_ex = np.reshape(train_X_ex, (2500, 800))
test_X_ex = np.reshape(test_X_ex, (500, 800))

train_y = np.array(train_y)
test_y = np.array(test_y)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators = 100, learning_rate = 1.0, random_state = 42)
clf.fit(train_X_ex, train_y)
clf.score(test_X_ex, test_y)

0.42

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
g_clf = GradientBoostingClassifier(n_estimators = 100, learning_rate=0.1, max_depth=1, random_state = 42)
g_clf.fit(train_X_ex, train_y)
g_clf.score(test_X_ex, test_y)

0.73